In [1]:
import pandas as pd
import numpy as np
import nsepy 
import os
from datetime import datetime ,date,timedelta
import datetime
from dateutil.relativedelta import *
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 1000)
%matplotlib inline


In [2]:
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 4, 30)
symbol = 'BANKNIFTY'

In [35]:
def getMonthlyExpirtyDate():
        local_start_date = start_date
        monthly_expirty_Date  =[]
        #start_date = datetime.date(2021, 1, 1)
        #end_date = datetime.date(2021, 4, 30)
        #delta = datetime.timedelta(days=1)
        delta = relativedelta(months=+1)
        while local_start_date <= end_date:
            #print(start_date.year,start_date.month)
            #print(str(max(nsepy.get_expiry_date(start_date.year,start_date.month))))

            monthly_expirty_Date.append(str(max(nsepy.get_expiry_date(local_start_date.year,local_start_date.month))))
            local_start_date += delta
        return monthly_expirty_Date
    
def checkfirstTradyingDayofMonth(first_day_of_the_month,expiry_day):
    for i in range(1,6):

        nifty_fut = nsepy.get_history(symbol=symbol,
                                    start=first_day_of_the_month,
                                    end=first_day_of_the_month,
                                    index=True,
                                    futures=True,
                                    expiry_date=expiry_day)
        if not nifty_fut.empty :

            return first_day_of_the_month
        else:
            #print(first_day_of_the_month)
            #print(type(first_day_of_the_month))
            first_day_of_the_month = (first_day_of_the_month+timedelta(days =1))
            #print("After",first_day_of_the_month)
            #print(type(first_day_of_the_month))

def getMonthlyExpirtyFromBahvcopy():
    database = "C:/Users/sanjo/PycharmProjects/mokshtech/stock_predictor/database/optionData"
    print(os.getcwd())
    print(os.listdir())
    local_start_date = start_date
    banknifty = pd.DataFrame()  
    monthly_expirty_Date  =[]
    delta = relativedelta(years=+1)

    try:
        while local_start_date <= end_date:
            val =0
            #print(str(symbol)+str(local_start_date.year)+".csv")
            filename = str(symbol)+str(local_start_date.year)+".csv"
            fname = os.path.join(database,filename)
            fname_day = os.path.join(database,filename)
            df = pd.read_csv(fname)
            df = df.loc[df['SYMBOL']== symbol]
            df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT']).dt.date

            df = df[df['EXPIRY_DT'] < end_date]
            df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT']).dt.strftime('%Y-%m-%d')
            df['EXPIRY_DT_Year_month'] = pd.to_datetime(df['EXPIRY_DT']).dt.strftime('%Y-%m')

            df1 = df[['EXPIRY_DT','EXPIRY_DT_Year_month']]

            banknifty= banknifty.append(df1)        
            local_start_date += delta  
    except Exception as error:
        print("File not found {0}".format(error)) 
    banknifty.sort_values("EXPIRY_DT", inplace = True)
    banknifty.drop_duplicates(inplace = True)   
    #for name , group in banknifty.groupby('EXPIRY_DT_Year_month'):
    #print(type(sorted(list(banknifty.groupby('EXPIRY_DT_Year_month').tail(1).EXPIRY_DT))))

    option_expiry_date_list = sorted(list(banknifty.groupby('EXPIRY_DT_Year_month').tail(1).EXPIRY_DT))
    #option_expiry_date_list =list(zip(*map(banknifty.get, ['EXPIRY_DT_Year_month','EXPIRY_DT'])))
    #print("option_expiry_date_list",option_expiry_date_list)

    return option_expiry_date_list

def cagr(start_value, end_value, num_periods):
     if num_periods >1:
         return (end_value / start_value) ** (1 / (num_periods - 1)) - 1
     else:
        return (end_value / start_value)
    
def getVixData(tradedate):
    vix_start_date = datetime.date(2015, 1, 1)
    vix = nsepy.get_history(symbol="INDIAVIX",
            start=vix_start_date,
            end=end_date,
            index=True)
    vix.dropna(inplace =True)

    vix['SMA'] = vix['Close'].rolling(window=20).mean()
    vix['date'] = vix.index
    #print(vix)
    vix['date'] = pd.to_datetime(vix['date'])
    
    
    return vix[vix['date']==tradedate]['SMA'].item()
            

In [4]:
expiry_list = getMonthlyExpirtyFromBahvcopy()

C:\Users\sanjo\PycharmProjects\mokshtech\stock_predictor\optionvaluecalculation\strategies
['.ipynb_checkpoints', 'BANKNIFTYmonthly_option_buy_Bhavcopy_Pandl.csv', 'BANKNIFTYmonthly_option_buy_nsepy_Pandl.csv', 'df.csv', 'forwordvolatility.py', 'input.csv', 'monthlyoptionbuying-addingsellleg.ipynb', 'monthlyoptionbuying.ipynb', 'monthy_nifty_option_buy_Bhavcopy_Pandl.csv', 'monthy_nifty_option_buy_nsepy_Pandl.csv', 'NIFTY.csv', 'optionPriceTrendAnalysis.ipynb', 'preDefinedStrategies', 'straddle.py', 'strategies.py', 'util.ipynb', 'util.py', 'weeklyOptionBuy.ipynb', 'weeklyOptionBuy_Selladditon.ipynb', 'weekly_banknifty_option_buy_Pandl.csv', 'weekly_nifty_option_buy_Pandl.csv', '__init__.py']


In [5]:
def getMonthlyOptionPriceTrend():
    database = "C:/Users/sanjo/PycharmProjects/mokshtech/stock_predictor/database/optionData"
    print(os.getcwd())
    print(os.listdir())
    
    banknifty = pd.DataFrame()  
    monthly_expirty_Date  =[]
    delta = relativedelta(years=+1)
    local_start_date = start_date
    symbol = 'BANKNIFTY'

    try:
        while local_start_date <= end_date:
            val =0
            #print(str(symbol)+str(local_start_date.year)+".csv")
            filename = str(symbol)+str(local_start_date.year)+".csv"
            fname = os.path.join(database,filename)
            fname_day = os.path.join(database,filename)
            df = pd.read_csv(fname)
            df['Date']=  pd.to_datetime(df['Date'])
            df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT'])
            banknifty= banknifty.append(df)        
            local_start_date += delta  
    except Exception as error:
        print("File not found {0}".format(error)) 
    return banknifty    




In [34]:
current_expiry_date= "2020-01-30"
expiry_day = datetime.datetime.strptime(current_expiry_date,'%Y-%m-%d')
print(expiry_day)
getVixData(expiry_day)

2020-01-30 00:00:00
               Open   High    Low    Close  Previous  Change  %Change      SMA       date
Date                                                                                     
2020-01-30  16.4975  17.14  15.38  16.7975   16.4975     0.3   0.0182  15.1615 2020-01-30


Date
2020-01-30    15.1615
Name: SMA, dtype: float64